<h3>1. Extracting data html table to the data frame<h3>

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from unicodedata import normalize

table_tor = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [5]:
print(f'Total tables: {len(table_tor)}')

Total tables: 3


In [6]:
table_tor = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', match='Borough')
len(table_tor)

1

In [7]:
df = table_tor[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [8]:
df.shape

(180, 3)

In [9]:
df.drop(df.loc[df['Borough']=='Not assigned'].index, inplace=True)

In [10]:
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [11]:
df.drop(df.loc[df['Neighbourhood']=='Not assigned'].index, inplace=True)

In [12]:
df.shape

(103, 3)

<h3>2. Retrieve Latitude/Longitude with "Geopy"<h3>

In [42]:
!conda install -c conda-forge geocoder --yes

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: | 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
                                                                                  |                                        failed

UnsatisfiableError: The following specifications were found
to be incompatible with the existing python installation in your environment:

Specifications:

  - cffi -> python[version='2.7.*|3.5.*|3.6.*|3.6.12|3.6.12|>=3.6,<3.7.0a0|>=3.7,<3.8.0a0|>=3.9,<3.10.0a0|>=3.8,<3.9.0a0|3.7.9|3.6.9|3.6.9|3.6.9|>=2.7,<2.8.0a0|3.6.9|>=3.5,<3.6.0a0|3.4.*',build='0_73_pypy|3_73_pypy|5_73_pypy|4_73_pypy|2_73_pypy|1_73_pypy']
  - rsa -> python[version='2.7.*|3.4.*|3.5.*|3.

In [13]:
 !pip install geocoder

In [20]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

KeyboardInterrupt: 

In [18]:
df = df.groupby(['Postal Code', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()

df.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [22]:
locgeo_df = pd.read_csv('https://cocl.us/Geospatial_data', index_col='Postal Code')

toronto_data = df.join(locgeo_df, on='Postal Code')

In [24]:
toronto_data.head(12)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


<h3>3. Clustering the neighborhoods in Toronto<h3>

In [30]:
!pip install folium

     |████████████████████████████████| 94 kB 5.3 MB/s  eta 0:00:01


In [31]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import json
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [34]:
geolocator = Nominatim(user_agent="coursera")
address = 'Toronto'
try:
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinates of {} are {}, {}.'.format(address, latitude, longitude))
except AttributeError:
    print('Cannot find: {}, will drop index: {}'.format(address, index))

my_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Postal Code']):
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(my_map)  
    
my_map

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


In [35]:
# The code was removed by Watson Studio for sharing.

let's take York(M6E)

In [37]:
toronto_data.set_index('Postal Code', inplace = True) 
neighborhood_latitude = toronto_data.loc['M6E']['Latitude']
neighborhood_longitude = toronto_data.loc['M6E']['Longitude']

create URL

In [39]:
# The code was removed by Watson Studio for sharing.

'https://api.foursquare.com/v2/venues/explore?&client_id=4MYN1GFRCK1DK03V0A2YNKT3YVUE1ZTEW52V1KDFCOUUITZS&client_secret=LYFIYCZSBFLJKENT2TPHPQLFTJWLDDPVKMJQVNF4ZOEJ0DPK&v=20180604&ll=43.6890256,-79.453512&radius=500&limit=100'

In [40]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6022813b56a5eb5a708ab122'},
 'response': {'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.6935256045, 'lng': -79.44730040297749},
   'sw': {'lat': 43.6845255955, 'lng': -79.45972359702252}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4eb0ca53be7bb9c3eeace0d6',
       'name': 'Nairn Park',
       'location': {'address': 'Nairn Ave',
        'crossStreet': 'Chudleigh Rd',
        'lat': 43.69065390931841,
        'lng': -79.4562995791835,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.69065390931841,
          'lng': -79.4562995791835}],
        'distance': 288,
        'cc': 'CA',
        'city': 

In [41]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [43]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [44]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Nairn Park,Park,43.690654,-79.456300
1,Maximum Woman,Women's Store,43.690651,-79.456333
2,Fairbanks Pool,Pool,43.691959,-79.448922
3,Fairbank Memorial Park,Park,43.692028,-79.448924


In [45]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


In [46]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [48]:
venues = getNearbyVenues(names=toronto_data['Borough'],latitudes=toronto_data['Latitude'],longitudes=toronto_data['Longitude'])

Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
East York
East York
East Toronto
East York
East York
East York
East Toronto
East Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
North York
York
York
Downtown Toronto
Wes

In [49]:
print(venues.shape)
venues.head()

(2107, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Scarborough,43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,Scarborough,43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,Scarborough,43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
3,Scarborough,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,Scarborough,43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank


In [50]:
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Central Toronto,108,108,108,108,108,108
Downtown Toronto,1204,1204,1204,1204,1204,1204
East Toronto,121,121,121,121,121,121
East York,76,76,76,76,76,76
Etobicoke,71,71,71,71,71,71
Mississauga,15,15,15,15,15,15
North York,250,250,250,250,250,250
Scarborough,91,91,91,91,91,91
West Toronto,155,155,155,155,155,155


In [51]:
print('There are {} uniques categories.'.format(len(venues['Venue Category'].unique())))

There are 266 uniques categories.


In [52]:
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = venues['Neighborhood'] 

onehot.head()

,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
onehot.shape

(2107, 266)

In [54]:
grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped

,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.009259,...,0.000000,0.000000,0.000000,0.009259,0.000000,0.000000,0.000000,0.000000,0.0000,0.009259
1,Downtown Toronto,0.000,0.000831,0.000831,0.000831,0.000831,0.001661,0.002492,0.001661,0.014120,...,0.002492,0.011628,0.001661,0.003322,0.000000,0.006645,0.000831,0.000000,0.0000,0.005814
2,East Toronto,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024793,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.024793
3,East York,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.013158,0.000000,0.000000,0.000000,0.0000,0.013158
4,Etobicoke,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014085,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014085,0.0000,0.000000
5,Mississauga,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
6,North York,0.008,0.000000,0.004000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008000,...,0.000000,0.000000,0.004000,0.008000,0.000000,0.000000,0.000000,0.000000,0.0120,0.000000
7,Scarborough,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010989,...,0.000000,0.000000,0.000000,0.010989,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
8,West Toronto,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.019355,0.000000,0.006452,0.000000,0.006452,0.000000,0.000000,0.0000,0.012903
9,York,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0625,0.000000


In [55]:
grouped.shape

(10, 266)

In [56]:
num_top_venues = 5

for hood in grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = grouped[grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
            venue  freq
0     Coffee Shop  0.08
1            Café  0.06
2            Park  0.06
3  Sandwich Place  0.06
4     Pizza Place  0.06


----Downtown Toronto----
                 venue  freq
0          Coffee Shop  0.11
1                 Café  0.05
2  Japanese Restaurant  0.03
3                Hotel  0.03
4           Restaurant  0.03


----East Toronto----
                venue  freq
0         Coffee Shop  0.07
1    Greek Restaurant  0.06
2             Brewery  0.04
3  Italian Restaurant  0.04
4          Restaurant  0.03


----East York----
                 venue  freq
0          Coffee Shop  0.05
1                 Bank  0.05
2  Sporting Goods Shop  0.04
3         Burger Joint  0.04
4         Intersection  0.04


----Etobicoke----
            venue  freq
0     Pizza Place  0.10
1        Pharmacy  0.06
2  Sandwich Place  0.06
3     Coffee Shop  0.06
4             Gym  0.04


----Mississauga----
                 venue  freq
0          Coffee Shop  0.20
1 

In [57]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [58]:
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Park,Pizza Place,Café
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant
2,East Toronto,Coffee Shop,Greek Restaurant,Brewery,Italian Restaurant,Ice Cream Shop
3,East York,Coffee Shop,Bank,Burger Joint,Intersection,Sporting Goods Shop
4,Etobicoke,Pizza Place,Sandwich Place,Pharmacy,Coffee Shop,Gym
5,Mississauga,Coffee Shop,Hotel,Fried Chicken Joint,American Restaurant,Middle Eastern Restaurant
6,North York,Coffee Shop,Clothing Store,Restaurant,Japanese Restaurant,Park
7,Scarborough,Bank,Bakery,Coffee Shop,Fast Food Restaurant,Breakfast Spot
8,West Toronto,Café,Bar,Bakery,Coffee Shop,Restaurant
9,York,Park,Sandwich Place,Breakfast Spot,Women's Store,Bar


In [59]:
# set number of clusters
kclusters = 5

grouped_clustering = grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 2, 0, 4, 3, 2, 0, 2, 1], dtype=int32)

In [60]:
merged = grouped

# add clustering labels
merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged = merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

merged.head() # check the last columns!

,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Central Toronto,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.009259,...,0.000000,0.000000,0.0,0.009259,2,Coffee Shop,Sandwich Place,Park,Pizza Place,Café
1,Downtown Toronto,0.0,0.000831,0.000831,0.000831,0.000831,0.001661,0.002492,0.001661,0.014120,...,0.000831,0.000000,0.0,0.005814,2,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant
2,East Toronto,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024793,...,0.000000,0.000000,0.0,0.024793,2,Coffee Shop,Greek Restaurant,Brewery,Italian Restaurant,Ice Cream Shop
3,East York,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.013158,0,Coffee Shop,Bank,Burger Joint,Intersection,Sporting Goods Shop
4,Etobicoke,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014085,...,0.000000,0.014085,0.0,0.000000,4,Pizza Place,Sandwich Place,Pharmacy,Coffee Shop,Gym


In [67]:
def get_latlon(postal_code):

    lat_lng_coords = None

    while(lat_lng_coords is None):

        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))

        lat_lng_coords = g.latlng

    return lat_lng_coords

get_latlon('M9B')   #  test function

[43.65034000000003, -79.55361999999997]

In [81]:
toronto_map = folium.Map(location=[43.65034000000003, -79.55361999999997], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_data['Latitude'], toronto_data['Longitude'], merged['Neighborhood'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [83]:
merged.loc[merged['Cluster Labels'] == 0, merged.columns[[1] + list(range(5,merged.shape[1]))]]

,Accessories Store,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,...,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.013158,0,Coffee Shop,Bank,Burger Joint,Intersection,Sporting Goods Shop
7,0.0,0.0,0.0,0.0,0.0,0.010989,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0,Bank,Bakery,Coffee Shop,Fast Food Restaurant,Breakfast Spot


In [84]:
merged.loc[merged['Cluster Labels'] == 1, merged.columns[[1] + list(range(5,merged.shape[1]))]]

,Accessories Store,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,...,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0625,0.0,1,Park,Sandwich Place,Breakfast Spot,Women's Store,Bar


In [85]:
merged.loc[merged['Cluster Labels'] == 2, merged.columns[[1] + list(range(5,merged.shape[1]))]]

,Accessories Store,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,...,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,0.000,0.000000,0.000000,0.000000,0.000000,0.009259,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000,0.009259,2,Coffee Shop,Sandwich Place,Park,Pizza Place,Café
1,0.000,0.000831,0.001661,0.002492,0.001661,0.014120,0.001661,0.004153,0.009967,0.001661,...,0.000831,0.0,0.000,0.005814,2,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant
2,0.000,0.000000,0.000000,0.000000,0.000000,0.024793,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000,0.024793,2,Coffee Shop,Greek Restaurant,Brewery,Italian Restaurant,Ice Cream Shop
6,0.008,0.000000,0.000000,0.000000,0.000000,0.008000,0.000000,0.000000,0.004000,0.000000,...,0.000000,0.0,0.012,0.000000,2,Coffee Shop,Clothing Store,Restaurant,Japanese Restaurant,Park
8,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.006452,0.000000,0.006452,0.000000,...,0.000000,0.0,0.000,0.012903,2,Café,Bar,Bakery,Coffee Shop,Restaurant


In [86]:
merged.loc[merged['Cluster Labels'] == 3, merged.columns[[1] + list(range(5,merged.shape[1]))]]

,Accessories Store,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,...,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
5,0.0,0.0,0.0,0.0,0.0,0.066667,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3,Coffee Shop,Hotel,Fried Chicken Joint,American Restaurant,Middle Eastern Restaurant


In [87]:
merged.loc[merged['Cluster Labels'] == 4, merged.columns[[1] + list(range(5,merged.shape[1]))]]

,Accessories Store,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,...,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
4,0.0,0.0,0.0,0.0,0.0,0.014085,0.0,0.0,0.0,0.0,...,0.0,0.014085,0.0,0.0,4,Pizza Place,Sandwich Place,Pharmacy,Coffee Shop,Gym
